In [1]:
!pip install catboost

     |████████████████████████████████| 63.9MB 44kB/s 


In [2]:
!pip install lightgbm

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
encoder = LabelEncoder()
scaler= StandardScaler()
import warnings 
from xgboost.sklearn import XGBRegressor
from xgboost.sklearn import XGBClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from catboost import CatBoostRegressor
from catboost import CatBoostClassifier

from lightgbm import LGBMRegressor
from lightgbm import LGBMClassifier





warnings.simplefilter('ignore')

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [10]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"aziz69","key":"6e8488da7a3e3ae164d88aa70c9dbc11"}'}

In [11]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [12]:
!kaggle competitions download -c ai-mini-hackathon-ept

  0% 0.00/3.42k [00:00<?, ?B/s]
100% 3.42k/3.42k [00:00<00:00, 5.92MB/s]
  0% 0.00/2.60k [00:00<?, ?B/s]
100% 2.60k/2.60k [00:00<00:00, 2.31MB/s]
  0% 0.00/1.61k [00:00<?, ?B/s]
100% 1.61k/1.61k [00:00<00:00, 1.66MB/s]
  0% 0.00/2.22M [00:00<?, ?B/s]
100% 2.22M/2.22M [00:00<00:00, 73.5MB/s]
  0% 0.00/7.98k [00:00<?, ?B/s]
100% 7.98k/7.98k [00:00<00:00, 9.06MB/s]


In [0]:
def custom_to_numeric(object):
        object = object.replace(',', '.')
        return pd.to_numeric(object)
converter={
        'Demande' : custom_to_numeric
}

In [0]:
data_train = pd.read_csv("training.csv",sep=";",index_col='Id', converters = converter)

In [0]:
data_test = pd.read_csv("test.csv",sep=";",index_col='Id',converters=converter)

In [0]:
test = pd.read_csv("score.csv",sep=";",index_col='Id')

In [17]:
data_train.head()

,MilestoneId,Prix,Demande
Id,,,
1,1,166,1.657704
2,1,165,1.883401
3,1,164,2.660588
4,1,163,3.056502
5,1,162,3.254603


In [18]:
data_train.isnull().sum()

MilestoneId    0
Prix           0
Demande        0
dtype: int64

In [19]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 367 entries, 1 to 524
Data columns (total 3 columns):
MilestoneId    367 non-null int64
Prix           367 non-null int64
Demande        367 non-null float64
dtypes: float64(1), int64(2)
memory usage: 11.5 KB


In [20]:
data_train.describe()

,MilestoneId,Prix,Demande
count,367.000000,367.000000,367.000000
mean,2.514986,101.634877,3262.778544
std,1.110885,38.522475,2803.106028
min,1.000000,36.000000,1.657704
25%,2.000000,68.000000,525.613764
50%,3.000000,101.000000,2716.607451
75%,3.000000,135.000000,6426.742216
max,4.000000,166.000000,7902.964756


In [21]:
data_train.shape

(367, 3)

In [23]:
frames = [data_train,data_test]
big_data = pd.concat(frames)
big_data.head()

,MilestoneId,Prix,Demande
Id,,,
1,1,166,1.657704
2,1,165,1.883401
3,1,164,2.660588
4,1,163,3.056502
5,1,162,3.254603


In [0]:
test1 = pd.read_csv('score.csv',sep=';')

In [27]:
test1.head()

,Id,MilestoneId,Prix
0,1,1,166
1,5,1,162
2,6,1,161
3,12,1,155
4,15,1,152


In [28]:
xgb = XGBRegressor()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:gamma'],
              'learning_rate': [0.04], #so called `eta` value
              'max_depth': [2],
              'min_child_weight': [5],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
           'n_estimators': [400,500]}

xgb_grid = GridSearchCV(xgb,
                        parameters,
                        cv = 3,
                        n_jobs = 5,
                        verbose=True
                       )

xgb_grid.fit(big_data.drop('Demande',axis=1),
         big_data['Demande'])

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   4 out of   6 | elapsed:    1.0s remaining:    0.5s
[Parallel(n_jobs=5)]: Done   6 out of   6 | elapsed:    1.1s finished


0.995395377990823
{'colsample_bytree': 0.7, 'learning_rate': 0.04, 'max_depth': 2, 'min_child_weight': 5, 'n_estimators': 500, 'nthread': 4, 'objective': 'reg:gamma', 'silent': 1, 'subsample': 0.7}


In [0]:
predictions = xgb_grid.predict(test)
grid1_pred = pd.DataFrame(predictions,columns=['Demande'])
grid1_pred.insert(0,'Id',test1['Id'])
grid1_pred.to_csv("MysubmissionX.csv",index=False)

In [30]:
xgb_grid.score(data_test.drop('Demande',axis=1),data_test['Demande'])

0.9956376083593705

In [0]:
y_predicted=xgb_grid.predict(data_test.drop('Demande',axis=1))

In [32]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(data_test['Demande'], y_predicted))
print(rms)

173.35623364722662


In [0]:

from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
encoder = LabelEncoder()
scaler1= StandardScaler()
scaler2=MinMaxScaler()
pt = PowerTransformer()
scaler3 = Normalizer()

x_train = scaler1.fit_transform(data_train.drop('Demande',axis=1))
x_test = scaler1.transform(data_test.drop('Demande',axis=1))

x_train1 = scaler2.fit_transform(data_train.drop('Demande',axis=1))
x_test1 = scaler2.transform(data_test.drop('Demande',axis=1))

x_train2 = pt.fit_transform(data_train.drop('Demande',axis=1))
x_test2 = pt.transform(data_test.drop('Demande',axis=1))

x_train3 = scaler3.fit_transform(data_train.drop('Demande',axis=1))
x_test3 = scaler3.transform(data_test.drop('Demande',axis=1))


data_train[['Prix']] = scaler2.fit_transform(data_train[['Prix']])
data_test[['Prix']] = scaler2.transform(data_test[['Prix']])

In [34]:
model = CatBoostRegressor()
parameters = {'depth'         : [8],
                  'learning_rate' : [0.03],
                  'iterations'    : [600]
                 }
grid1 = GridSearchCV(estimator=model, param_grid = parameters, cv = 3, n_jobs=-1)
grid1.fit(data_train.drop("Demande",axis=1), data_train['Demande'])    

    # Results from Grid Search
print("\n========================================================")
print(" Results from Grid Search " )
print("========================================================")    
    
print("\n The best estimator across ALL searched params:\n",
          grid1.best_estimator_)
    
print("\n The best score across ALL searched params:\n",
          grid1.best_score_)
    
print("\n The best parameters across ALL searched params:\n",
          grid1.best_params_)
    
print("\n ========================================================")

0:	learn: 2731.8245949	total: 46.4ms	remaining: 27.8s
1:	learn: 2660.0722017	total: 47.7ms	remaining: 14.3s
2:	learn: 2591.9113778	total: 48.1ms	remaining: 9.57s
3:	learn: 2525.4564577	total: 49.2ms	remaining: 7.33s
4:	learn: 2459.1164612	total: 49.5ms	remaining: 5.89s
5:	learn: 2396.1533491	total: 50.3ms	remaining: 4.98s
6:	learn: 2331.7279477	total: 50.8ms	remaining: 4.3s
7:	learn: 2275.0265428	total: 51.2ms	remaining: 3.79s
8:	learn: 2216.5557621	total: 51.6ms	remaining: 3.39s
9:	learn: 2161.3078985	total: 52.6ms	remaining: 3.1s
10:	learn: 2104.3087759	total: 53.2ms	remaining: 2.85s
11:	learn: 2048.6131874	total: 53.9ms	remaining: 2.64s
12:	learn: 1997.5955489	total: 54.2ms	remaining: 2.45s
13:	learn: 1944.2433975	total: 54.9ms	remaining: 2.3s
14:	learn: 1893.7166576	total: 55.8ms	remaining: 2.18s
15:	learn: 1843.4999007	total: 56.5ms	remaining: 2.06s
16:	learn: 1801.2636479	total: 56.8ms	remaining: 1.95s
17:	learn: 1753.5476373	total: 57.3ms	remaining: 1.85s
18:	learn: 1708.1546341

In [35]:
y_predicted=grid1.predict(data_test.drop("Demande",axis=1))
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(data_test['Demande'], y_predicted))
print(rms)

31.42382148661028


In [36]:


model = CatBoostRegressor()
parameters = {'depth'         : [8],
                  'learning_rate' : [0.03],
                  'iterations'    : [10000,20000]
                 }
grid1 = GridSearchCV(estimator=model, param_grid = parameters, cv = 3, n_jobs=-1)
grid1.fit(big_data.drop('Demande',axis=1), big_data['Demande'])    

    # Results from Grid Search
print("\n========================================================")
print(" Results from Grid Search " )
print("========================================================")    
    
print("\n The best estimator across ALL searched params:\n",
          grid1.best_estimator_)
    
print("\n The best score across ALL searched params:\n",
          grid1.best_score_)
    
print("\n The best parameters across ALL searched params:\n",
          grid1.best_params_)
    
print("\n ========================================================")

0:	learn: 2682.1579023	total: 488us	remaining: 9.77s
1:	learn: 2610.2602280	total: 1.4ms	remaining: 14s
2:	learn: 2544.2406606	total: 1.99ms	remaining: 13.3s
3:	learn: 2477.2476234	total: 3.33ms	remaining: 16.6s
4:	learn: 2410.3690231	total: 3.87ms	remaining: 15.5s
5:	learn: 2348.9028961	total: 4.44ms	remaining: 14.8s
6:	learn: 2285.1826728	total: 5.16ms	remaining: 14.7s
7:	learn: 2226.5400103	total: 6.46ms	remaining: 16.1s
8:	learn: 2168.2929278	total: 7.52ms	remaining: 16.7s
9:	learn: 2114.1673836	total: 7.89ms	remaining: 15.8s
10:	learn: 2057.1935097	total: 8.49ms	remaining: 15.4s
11:	learn: 2001.5781229	total: 9.47ms	remaining: 15.8s
12:	learn: 1952.3552428	total: 9.93ms	remaining: 15.3s
13:	learn: 1901.2152988	total: 10.9ms	remaining: 15.6s
14:	learn: 1848.2836026	total: 11.7ms	remaining: 15.5s
15:	learn: 1800.3234692	total: 12.7ms	remaining: 15.9s
16:	learn: 1753.2641282	total: 14.3ms	remaining: 16.8s
17:	learn: 1707.3074979	total: 15.3ms	remaining: 17s
18:	learn: 1661.6651592	to

In [0]:
predictions = grid1.predict(test)
grid1_pred = pd.DataFrame(predictions,columns=['Demande'])
grid1_pred.insert(0,'Id',test1['Id'])
grid1_pred.to_csv("MysubmissionX.csv",index=False)